In [11]:
import pandas as pd

In [12]:
path_predictions = "gpt-codes/codiesp-gpt4-fewshot-0-19.tsv"
path_gold = "codiesp/final_dataset_v4_to_publish/dev/devX.tsv"

header_X = ["FILE","TYPE", "CODE", "SUBSTRING", "OFFSETS"]


In [13]:
predictions = pd.read_csv(path_predictions, delimiter="\t", names=header_X)
gold = pd.read_csv(path_gold, delimiter="\t", names=header_X)

print(gold.head())
print(predictions.head())


                        FILE           TYPE     CODE  \
0  S0004-06142005000900016-1  PROCEDIMIENTO  bt41zzz   
1  S0004-06142005000900016-1  PROCEDIMIENTO     ct13   
2  S0004-06142005000900016-1    DIAGNOSTICO   q62.11   
3  S0004-06142005000900016-1    DIAGNOSTICO   n28.89   
4  S0004-06142005000900016-1    DIAGNOSTICO    n39.0   

                                     SUBSTRING          OFFSETS  
0                      ecografía renal derecha  307 316;348 361  
1                            gammagrafía renal          739 756  
2  estenosis en la unión pieloureteral derecha          540 583  
3                        ectasia pielocalicial          326 347  
4              infecciones del tracto urinario          198 229  
                        FILE           TYPE     CODE  \
0  S0004-06142005000900016-1    DIAGNOSTICO    k26.9   
1  S0004-06142005000900016-1    DIAGNOSTICO    k59.0   
2  S0004-06142005000900016-1    DIAGNOSTICO   r10.31   
3  S0004-06142005000900016-1    DIAGNOSTICO

In [14]:
files = predictions["FILE"].unique()
print(files)


['S0004-06142005000900016-1' 'S0004-06142005001000011-1'
 'S0004-06142006000200011-1' 'S0004-06142006000500002-3'
 'S0004-06142006000500002-4' 'S0004-06142006000600015-1'
 'S0004-06142006000700008-1' 'S0004-06142006000900008-1'
 'S0004-06142006000900009-1' 'S0004-06142007000600011-1'
 'S0004-06142007000700014-1' 'S0004-06142007000700015-1'
 'S0004-06142008000100008-1' 'S0004-06142008000300011-1'
 'S0004-06142008000300013-1' 'S0004-06142008000400013-1'
 'S0004-06142008000500014-1' 'S0004-06142008000500015-1']


In [15]:
gold = gold[gold["FILE"].isin(files)]

In [16]:
print(gold["FILE"].unique())

['S0004-06142005000900016-1' 'S0004-06142005001000011-1'
 'S0004-06142006000200011-1' 'S0004-06142006000500002-3'
 'S0004-06142006000500002-4' 'S0004-06142006000600015-1'
 'S0004-06142006000700008-1' 'S0004-06142006000900008-1'
 'S0004-06142006000900009-1' 'S0004-06142007000600011-1'
 'S0004-06142007000700014-1' 'S0004-06142007000700015-1'
 'S0004-06142008000100008-1' 'S0004-06142008000300011-1'
 'S0004-06142008000300013-1' 'S0004-06142008000400013-1'
 'S0004-06142008000500014-1' 'S0004-06142008000500015-1']


In [17]:
def eval_string(gold, predictions, file):
    gold_select = gold[gold["FILE"] == file]
    predictions_select = predictions[predictions["FILE"] == file]
    print(gold_select["SUBSTRING"].unique())
    print(predictions_select["SUBSTRING"].unique())

In [18]:
eval_string(gold, predictions, files[3])

['cólicos nefríticos de repetición' 'nódulo en testículo izquierdo'
 'Fumador' 'testículo derecho con masa' 'ecografía-doppler testicular'
 'orquiectomía radical inguinal derecha' 'TAC toraco-abdómino-pélvico'
 'Rx de tórax' 'azoospermia' 'biopsia del teste izquierdo'
 'atrofia testicular' 'metástasis' 'cólicos nefríticos' 'tumor testicular'
 'quiste de epidídimo' 'Uretrotomía' 'neoplasia de testículo'
 'biopsia del testículo izquierdo vía inguinal'
 'biopsia del testículo izquierdo' 'testículo quiste' 'testículo atrofia'
 'criptorquidia bilateral']
['atrofia testicular' 'azoospermia total'
 'orquiectomía radical inguinal derecha' 'quiste de epidídimo'
 'nódulo en testículo izquierdo' 'quiste de cordón izquierdo'
 'TAC toraco-abdómino-pélvico' 'Rx de tórax' 'cólicos nefríticos'
 'neoplasia de testículo' 'tumor testicular' 'Tumor de células de Leydig'
 'esterilidad' 'Ureterolitectomía' 'cólicos nefríticos de repetición'
 'tratamiento con LEOC' 'criptorquidia bilateral' 'orquidopexia'
 '

In [10]:
def get_prompt1(input):
    prompt = """Extract the list of substrings that cover the procedures and diagnoses for an input text. The substrings should form the basis for ICD-10 coding. I will give one example. Focus on recall over precision, repeat substrings with multiple occurences. Your output format is only ["extracted substring"] form, no other form.
    
    <!-- begin example -->
    Mujer de 29 años con antecedentes de ulcus duodenal y estreñimiento que consulta por dolor en fosa renal derecha compatible con crisis renoureteral. No antecedentes de nefrolitiasis ni hematuria ni infecciones del tracto urinario. En la exploración sólo destaca una puñopercusión renal derecha positiva. La ecografía objetiva ectasia pielocalicial renal derecha con adelgazamiento del parénquima. La UIV muestra una anulación funcional de la unidad renal derecha siendo normal el resto de la exploración. Una pielografía retrograda muestra estenosis en la unión pieloureteral derecha, siendo la citología urinaria selectiva del uréter derecho negativa. Ante la disyuntiva de practicar una cirugía reconstructiva o una exerética se realiza gammagrafía renal que demuestra captación relativa del 33% para el riñón derecho y del 67% para el izquierdo.

    Se realiza pieloplastia renal derecha según técnica de Anderson-Hynes.
    El examen anatomopatológico de la estenosis de la vía urinaria descubre un hemangioma cavernoso como causante de la obstrucción, el cual no permitía pasar ninguna cánula. El control postoperatorio urográfico a los 3 meses objetiva captación y eliminación renal derecha de contraste con ligero retraso respecto a riñón izquierdo.

    La paciente fue dada de alta de consultas externas a los tres años de la cirugía tras haber sido practicados otros dos controles urográficos; 7 años después de la pieloplastia no ha vuelto a consultar por clínica urológica.

    Output:
    ['ecografía renal derecha' 'gammagrafía renal'
    'estenosis en la unión pieloureteral derecha' 'ectasia pielocalicial'
    'infecciones del tracto urinario' 'hematuria' 'dolor en fosa renal'
    'crisis renoureteral' 'unión pieloureteral estenosis'
    'urinaria hemangioma cavernoso' 'estenosis en la unión pieloureteral'
    'ulcus duodenal' 'anulación funcional de la unidad renal' 'nefrolitiasis'
    'estreñimiento']
    <!-- end example -->

    {input}

    Output:
    """.format(input=input)
    return prompt

print(get_prompt1(input="test"))

Extract the list of substrings that cover the procedures and diagnoses for an input text. The substrings should form the basis for ICD-10 coding. I will give one example. Focus on recall over precision, repeat substrings with multiple occurences. Your output format is only ["extracted substring"] form, no other form.
    
    <!-- begin example -->
    Mujer de 29 años con antecedentes de ulcus duodenal y estreñimiento que consulta por dolor en fosa renal derecha compatible con crisis renoureteral. No antecedentes de nefrolitiasis ni hematuria ni infecciones del tracto urinario. En la exploración sólo destaca una puñopercusión renal derecha positiva. La ecografía objetiva ectasia pielocalicial renal derecha con adelgazamiento del parénquima. La UIV muestra una anulación funcional de la unidad renal derecha siendo normal el resto de la exploración. Una pielografía retrograda muestra estenosis en la unión pieloureteral derecha, siendo la citología urinaria selectiva del uréter derecho ne

In [19]:
gold.head()

,FILE,TYPE,CODE,SUBSTRING,OFFSETS
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307 316;348 361
1,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739 756
2,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis en la unión pieloureteral derecha,540 583
3,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia pielocalicial,326 347
4,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones del tracto urinario,198 229


In [ ]:
for 